# Instance-based metode

**"A family of learning algorithms that, instead of performing explicit generalization, compares new problem instances with instances seen in training"**

# Pregled
* Instance-based metode klasifikacije
* k-nearest neighbours (k-NN)
  * Weighted k-NN
* Lokalno-težinska regresija (LWR)

# Instance-based metode klasifikacije

**Instance-based metode** su metode koje ne grade eksplicitan model podataka (u vidu neke funkcije) koji generalizuje. Umesto toga, klasifikacija se vrši na osnovu skupa instanci za trening, koje se čuvaju i bivaju upotrebljene tek kada je potrebno da se izvrši sama klasifikacija.

# k-nearest neighbours
**k-nearest neighbours** tj. **k-NN** je tehnika koja klasifikuje nepoznate primere na osnovu k najbližih suseda (uglavnom po euklidskoj distanci u prostoru feature-a). Dakle, umesto treninga, nepoznati primer $x$ klasifikujemo u klasu koja se najčešće pojavljuje u najbližih k trening primera. Ovo čini da faza treninga bude nepostojeća, ali da faza predikcije za novi primer bude u nekim slučajevima veoma spora.

U **Weighted k-NN** varijaciji ne dobija glas svakog od k suseda istu težinu tj. uvodimo eksplicitne težine koje su obrnuto proporcionalne udaljenošću (uglavnom 1/d ako je udaljenost d).

Pored načina na koji se dodeljuju težine, jedini parametar modela je **k**. Ovaj parametar predstavlja balans između posmatranja lokalne i globalne strukture: ukoliko stavimo **k=1** model prestaje da bude otporan na šum tj. prisustvo outlier-a. Sa deuge strane, povećavanje **k** donosi bolje rezultate do neke granice, kada previše uzimamo u obzir globalna svojstva.

Jedna od čestih primena k-NN modela je u sistemima za preporuku (recommender systems) tj. u metodama kolaborativnog filtriranja (collaborative filtering). Npr. na osnovu skupa podataka koji sadrži ocene velikog broja ljudi za određene filmove, možemo novom korisniku preporučiti film koji bi bio odgovarajući.

U našem slučaju pogledaćemo k-NN u praksi na primeru klasifikacije ručno pisanih cifara ([MNIST](http://yann.lecun.com/exdb/mnist/) skup podataka).

## k-NN za MNIST

In [0]:
# TODO

# Lokalno-težinska regresija (LWR)

* **Lokalno-težinska regresija** (Locally-Weighted Regression, LWR)
* Umesto jedne, treniramo lokalno-težinsku regresiju za neki skup tačaka
* Za svaku odabranu tačku u formulaciji za MSE dajemo veću težinu bližim tačkama
* Težina se računa po određenoj formuli
  * Često: $w(i) = \exp({-\frac{(x^{(i)} - x)^2}{2\tau^2}})$
* [Više o LWR](https://see.stanford.edu/materials/aimlcs229/cs229-notes1.pdf)


In [0]:
# TODO